In [9]:
import numpy as np
import pandas as pd

In [10]:
pd.__version__

'2.0.3'

## Pandas基本概念
- 数据结构：Series(一维)，DataFrame(二维)，基于numpy开发
- 缺失值为NaN
- 用行(index)和列(columns)代替了numpy中的axis0和axis1
- 值可变，但结构变化有一定的限制
- 绝大多数方法都是复制数据在新对象上操作，保持原数据不变
- DataFrame每列数据类型都可以不同，可以通过DataFrame.to_numpy()转换为numpy对象，类型很多时很耗费系统资源

> 参考[pandas官方文档](https://www.pypandas.cn/docs/getting_started/10min.html)


In [11]:
index = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index,
                   columns=['A', 'B', 'C'])

### 文件读取
- pd.read_table('filepath', header, index_col, usecols, parse_dates, nrows)
- header第一行是否作为列名，index_col哪些列作为索引，usecols读取多少列
- parse_dates哪列要转化为时间序列，nrows读取多少行
- pd.read_table('../xx.txt', sep=' \|\|\|\| ', engine='python') 读取txt时遇到特殊分隔符，可以用sep正则指定

### 文件写入
- df_txt.to_csv('../xx.txt', sep='\t', index=False) 用这种形式保存为txt
- 当索引没有特殊意义的时候index设置为False，以去除索引

## 基本数据结构
- object代表了一种混合类型,有多种类型时则为object

Series
- Series一般由四个部分组成，分别是序列的值data、索引index、存储类型dtype、序列的名字name

In [12]:
s = pd.Series(data = [100, 'a', {'dic1':5}],
              index = pd.Index(['id1', 20, 'third'], name='my_idx'),
              dtype = 'object',
              name = 'my_name')
s

my_idx
id1              100
20                 a
third    {'dic1': 5}
Name: my_name, dtype: object

In [13]:
s.shape

(3,)

DataFrame
- 在series的基础上增加了列索引
- 一般通过列索引名到数据的映射来构造数据框，同时再加上行索引

In [14]:
data = [[1, 'a', 1.2], [2, 'b', 2.2], [3, 'c', 3.2]]
df = pd.DataFrame(data = data,
                  index = ['row_%d'%i for i in range(3)],
                  columns=['col_0', 'col_1', 'col_2'])
df

,col_0,col_1,col_2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


In [37]:
# 常用构造方法
df = pd.DataFrame(data = {'col_0': [1,2,3],
                          'col_1':list('abc'),
                          'col_2': [1.2, 2.2, 3.2]},
                  index = ['row_%d'%i for i in range(3)])
df

,col_0,col_1,col_2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


# 常用函数

汇总函数
- head, tail函数分别表示返回表或者序列的前n行和后n行，n默认为5
- info, describe分别返回表的信息概况和表中数值列对应的主要统计量

In [16]:
df.head(1)

,col_0,col_1,col_2
row_0,1,a,1.2


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, row_0 to row_2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col_0   3 non-null      int64  
 1   col_1   3 non-null      object 
 2   col_2   3 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 96.0+ bytes


统计函数
- sum, mean, median, var, std, max, min
- quantile 分位数
- count 非缺失值个数
- idxmax 最大值对应的索引
- 可以修改axis选择按行还是按列

In [18]:
df['col_0'].mean()

2.0

In [19]:
df['col_0'].quantile(0.3)

1.6

In [20]:
df['col_0'].idxmax()

'row_2'

In [21]:
df['col_0'].count()

3

唯一值函数
- unique和nunique 唯一值组成的列表和唯一值的个数
- value_counts 唯一值和其对应出现的频数
- drop_duplicates 多个列组合的唯一值，keep=first表示每个组合保留第一次出现的所在行，last表示保留最后一次出现的所在行，False表示把所有重复组合所在的行剔除
- duplicated 是否为唯一值的布尔列表

In [22]:
df['col_0'].duplicated()

row_0    False
row_1    False
row_2    False
Name: col_0, dtype: bool

In [23]:
df['col_0'].nunique()

3

In [24]:
df['col_0'].value_counts()

col_0
1    1
2    1
3    1
Name: count, dtype: int64

替换函数
- 替换操作是针对某一个列进行的，可以归纳为三类：映射替换、逻辑替换、数值替换
- replace({'NAME1':V1, 'NAME2':V2})
- where和mask分别在条件为false和true时替换
- round, abs, clip方法，它们分别表示按照给定精度四舍五入、取绝对值和截断

In [25]:
df['col_0'].replace({'row_1':10})

row_0    1
row_1    2
row_2    3
Name: col_0, dtype: int64

In [26]:
df['col_0'].replace(['row_0'],[10])

row_0    1
row_1    2
row_2    3
Name: col_0, dtype: int64

In [27]:
# method控制方向替换
s = pd.Series(['a', 1, 'b', 2, 1, 1, 'a'])
s.replace([1, 2], method='ffill')

0    a
1    a
2    b
3    b
4    b
5    b
6    a
dtype: object

In [28]:
# 逻辑替换
s = pd.Series([-1, 1.2345, 100, -50])
s.where(s<0)

0    -1.0
1     NaN
2     NaN
3   -50.0
dtype: float64

In [29]:
s.where(s<0, 100)

0     -1.0
1    100.0
2    100.0
3    -50.0
dtype: float64

In [30]:
s.mask(s<0, -50)

0    -50.0000
1      1.2345
2    100.0000
3    -50.0000
dtype: float64

In [31]:
s = pd.Series([-1, 1.2345, 100, -50])
s.round(2)

0     -1.00
1      1.23
2    100.00
3    -50.00
dtype: float64

In [32]:
s.abs()

0      1.0000
1      1.2345
2    100.0000
3     50.0000
dtype: float64

In [33]:
s.clip(0, 2) # 前两个数分别表示上下截断边界

0    0.0000
1    1.2345
2    2.0000
3    0.0000
dtype: float64

排序函数
- sort_values值排序
- sort_index索引排序
- 默认参数ascending=True为升序

In [ ]:
# 同时对多列排序，体重相同时身高降序
df_demo.sort_values(['Weight','Height'],ascending=[True,False]).head()

# 索引排序多个level即可
df_demo.sort_index(level=['Grade','Name'],ascending=[True,False]).head()

# 窗口对象

滑窗对象
- 对一个序列使用.rolling得到滑窗对象，然后对滑窗对象使用聚合函数
- shift 取向前第n个元素的值
- diff 与向前第n个元素做差
- pct_change 与向前第n个元素相比计算增长率
- 公共参数为periods=n，默认为1。这里的n可以为负，表示反方向的类似操作

In [40]:
s = pd.Series([1,2,3,4,5])
roller = s.rolling(window = 3)
roller

Rolling [window=3,center=False,axis=0,method=single]

In [41]:
roller.mean()

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

In [43]:
roller.corr()

0    NaN
1    NaN
2    1.0
3    1.0
4    1.0
dtype: float64

扩张窗口
- 聚合函数会作用于这些逐步扩张的窗口上

In [44]:
s = pd.Series([1, 3, 6, 10])
s.expanding().mean()

0    1.000000
1    2.000000
2    3.333333
3    5.000000
dtype: float64